In [94]:
import awswrangler as wr
import pandas as pd
from datetime import datetime

DATABASE_NAME = "DB_KELLY_v1"

def getDataFromDB(signal_list, startTime, endTime, resampling):
    count = 0
    for signal in signal_list:
        querry = f"SELECT time, measure_value::double FROM {DATABASE_NAME}.{signal} WHERE time BETWEEN TIMESTAMP '{startTime}' AND TIMESTAMP '{endTime}' ORDER BY time ASC"
        print(querry)
        df = wr.timestream.query(
            f"{querry}"
        )
        value_colName = list(df.columns)[1]
        # print(value_colName)
        df.rename(columns = {value_colName:signal}, inplace = True)
        df.time = pd.to_datetime(df.time)
        print('len df: ', len(df))
        df = df.set_index(['time'])

        resampling_str = str(resampling)
        df_resample = df.copy().resample(resampling_str).last().bfill(axis ='rows').reset_index()
        # df_resample = df.copy().resample(resampling_str).reset_index()
        # print('resample: ', len(df_resample))

        if count != 0:
            df_all = pd.merge_asof(df_all, df_resample, on='time')
        else:
            df_all = df_resample
        count = count+1

    return df_all

In [141]:
# first signal has to be any voltage signal
# tolerance is the range for nearest time, input should be in "1S", "100ms" etc.
# date format "yyyy-mm-dd hr:min:sec"
def getSyncedSamplesFromDB(signal_list, startTime, endTime, tolerance="300ms"):
    count = 0
    df_all = pd.DataFrame()
    for signal in signal_list:
        querry = f"SELECT time, measure_value::double FROM {DATABASE_NAME}.{signal} WHERE time BETWEEN TIMESTAMP '{startTime}' AND TIMESTAMP '{endTime}' ORDER BY time ASC"
        print(querry)
        try:
            df = wr.timestream.query(
                f"{querry}"
            )
        except:
            df = pd.DataFrame()      # create empty dataframe
            print("query returned no results")

        if (len(df) > 0):
            value_colName = list(df.columns)[1]
            df.rename(columns = {value_colName:signal}, inplace = True)
            df.time = pd.to_datetime(df.time)
        
            if count > 0:
                df_all = pd.merge_asof(df_all, df.copy(), on="time", direction='nearest', tolerance=pd.Timedelta("300ms"), allow_exact_matches=False)
            else:
                df_all = df.copy()       # first signal, keep as is
        else:
            print("no records found")
            break
        count = count+1

    return df_all

In [ ]:
# !pip install 
# 1. generator : fn to send input in batches
# 2. pipeline: seq of steps applied over data
# 3. model
# 4. Asif: dynamic model 
# model: histogram, binning, transformation (fourier)
# visualization of prediction vs real 
# hyperparam tuning discuss with Asif
# error matrix

In [2]:
start_date = '2022-04-03 11:58:00'
end_date = '2022-04-05 12:00:00'
sampling_rate = 100      # 100 ms
cols_module1 = ['current_BMS01',
'stringVoltage_BMS01',
'minSOC_BMS01',
'SOCocvInit_BMS01',
'minSOH_BMS01',
'contactorState_BMS01',
'ctrOffTime_BMS01',
'stringState_BMS01',
'dchgIntResistance_BMS01',
'stringStateReq_BMS01_BMS',
'brickVoltage001_BMS01',
'brickVoltage002_BMS01',
'brickVoltage003_BMS01',
'brickVoltage004_BMS01',
'brickVoltage005_BMS01',
'brickVoltage006_BMS01',
'brickVoltage007_BMS01',
'brickVoltage008_BMS01',
'brickVoltage009_BMS01',
'brickVoltage010_BMS01',
'brickVoltage011_BMS01',
'brickVoltage012_BMS01',
'moduleTemperature01_BMS01',
'moduleTemperature02_BMS01',
'moduleTemperature03_BMS01',
'moduleTemperature04_BMS01']

df_mod1 = getDataFromDB(cols_module1, start_date, end_date, sampling_rate)

SELECT time, measure_value::double FROM DB_KELLY_v1.current_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-03 11:58:00' AND TIMESTAMP '2022-04-05 12:00:00' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.stringVoltage_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-03 11:58:00' AND TIMESTAMP '2022-04-05 12:00:00' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.minSOC_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-03 11:58:00' AND TIMESTAMP '2022-04-05 12:00:00' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.SOCocvInit_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-03 11:58:00' AND TIMESTAMP '2022-04-05 12:00:00' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.minSOH_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-03 11:58:00' AND TIMESTAMP '2022-04-05 12:00:00' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.contactorState_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-03 11:58:00' AND TIMESTAMP 

In [6]:
# Save df with average temperature
import sys, os
cwd = sys.path[0]

dir_path = os.path.join(cwd, 'csv', "phase_3_pkl")
print(dir_path)
src_path = os.path.join(dir_path, 'module1_phase3.pkl')
print(src_path)
df_mod1.to_pickle(src_path)
len(df_mod1)
print(df_mod1.shape)
# 02 Apr onwards

/media/avt/development/ME_dev/battery_aging_algo/csv/phase_3_pkl
/media/avt/development/ME_dev/battery_aging_algo/csv/phase_3_pkl/module1_phase3.pkl
(1665167, 27)


In [13]:
df_mod1.head(100)
# df_all = df_all[df_all["minSOC_BMS01"] < 110]
# df_all = df_all[df_all["minSOC_BMS01"] > 5]
# df_all = df_all[df_all["brickVoltage001_BMS01"] < 5]
# df_all = df_all[df_all["brickVoltage001_BMS01"] > 3]
# df_all.plot(x ='time', y=df_all.columns[1:])
# df1 = df[:500000]
# df1.plot(x ='time', y='measure_value::double', kind='scatter', figsize=(15,3))

,time,current_BMS01,stringVoltage_BMS01,minSOC_BMS01,SOCocvInit_BMS01,minSOH_BMS01,contactorState_BMS01,ctrOffTime_BMS01,stringState_BMS01,dchgIntResistance_BMS01,...,brickVoltage007_BMS01,brickVoltage008_BMS01,brickVoltage009_BMS01,brickVoltage010_BMS01,brickVoltage011_BMS01,brickVoltage012_BMS01,moduleTemperature01_BMS01,moduleTemperature02_BMS01,moduleTemperature03_BMS01,moduleTemperature04_BMS01
0,2022-04-03 13:44:43.300,3353.5,338.64,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,6.5535,6.5535,6.5535,6.5535,6.5535,6.5535,NaN,215.0,215.0,215.0
1,2022-04-03 13:44:43.400,3353.5,338.64,655.35,0.0,NaN,0.0,NaN,0.0,NaN,...,3.5220,3.5221,3.5257,3.5263,3.5217,3.5215,NaN,15.0,15.0,15.0
2,2022-04-03 13:44:43.500,0.0,338.64,13.44,1.0,96.5,0.0,NaN,0.0,3276.75,...,3.5220,3.5221,3.5257,3.5263,3.5217,3.5215,NaN,15.0,15.0,15.0
3,2022-04-03 13:44:43.600,0.0,338.64,13.44,1.0,96.5,0.0,NaN,0.0,1.75,...,3.5220,3.5221,3.5257,3.5263,3.5217,3.5215,15.0,15.0,15.0,15.0
4,2022-04-03 13:44:43.700,0.0,338.64,13.44,1.0,96.5,0.0,65535.0,0.0,1.75,...,3.5220,3.5221,3.5258,3.5263,3.5217,3.5216,15.0,15.0,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-04-03 13:44:52.800,0.0,338.64,13.44,1.0,96.5,0.0,65535.0,0.0,1.75,...,3.5220,3.5220,3.5257,3.5262,3.5217,3.5215,15.0,15.0,15.0,15.0
96,2022-04-03 13:44:52.900,0.0,338.64,13.44,1.0,96.5,0.0,65535.0,0.0,1.75,...,3.5220,3.5220,3.5257,3.5262,3.5217,3.5215,15.0,15.0,15.0,15.0
97,2022-04-03 13:44:53.000,0.0,338.64,13.44,1.0,96.5,0.0,65535.0,0.0,1.75,...,3.5220,3.5220,3.5257,3.5262,3.5217,3.5215,15.0,15.0,15.0,15.0
98,2022-04-03 13:44:53.100,0.0,338.64,13.44,1.0,96.5,0.0,65535.0,0.0,1.75,...,3.5220,3.5220,3.5257,3.5262,3.5217,3.5215,15.0,15.0,15.0,15.0


In [151]:
# Get data
start_date = '2022-03-29 00:00:00'
end_date = '2022-06-01 23:23:59'

cols = ['brickVoltage001_BMS01', 'brickVoltage007_BMS01', 'brickVoltage015_BMS01', 'brickVoltage025_BMS01', 'current_BMS01', 'minSOC_BMS01', 'SOCocvInit_BMS01', 'minSOH_BMS01', 'contactorState_BMS01', 'ctrOffTime_BMS01',  'moduleTemperature01_BMS01']

df = getSyncedSamplesFromDB(cols, start_date, end_date)


SELECT time, measure_value::double FROM DB_KELLY_v1.brickVoltage001_BMS01 WHERE time BETWEEN TIMESTAMP '2022-03-29 00:00:00' AND TIMESTAMP '2022-06-01 23:23:59' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.brickVoltage007_BMS01 WHERE time BETWEEN TIMESTAMP '2022-03-29 00:00:00' AND TIMESTAMP '2022-06-01 23:23:59' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.brickVoltage015_BMS01 WHERE time BETWEEN TIMESTAMP '2022-03-29 00:00:00' AND TIMESTAMP '2022-06-01 23:23:59' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.brickVoltage025_BMS01 WHERE time BETWEEN TIMESTAMP '2022-03-29 00:00:00' AND TIMESTAMP '2022-06-01 23:23:59' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.current_BMS01 WHERE time BETWEEN TIMESTAMP '2022-03-29 00:00:00' AND TIMESTAMP '2022-06-01 23:23:59' ORDER BY time ASC
SELECT time, measure_value::double FROM DB_KELLY_v1.minSOC_BMS01 WHERE time BETWEEN TIMESTAMP '2022-03-29 00:00:0

In [153]:
print(df['current_BMS01'].min())
print(df['moduleTemperature01_BMS01'].min())
print(df['current_BMS01'].max())
print(df['moduleTemperature01_BMS01'].max())
print(df['brickVoltage001_BMS01'].min())
print(df['brickVoltage001_BMS01'].max())
print(len(df))

-333.1999999999998
1.49202829e-315
3353.5
215.0
0.0
215.0
2008037


In [154]:
# Save
import sys, os
cwd = sys.path[0]

dir_path = os.path.join(cwd, 'csv', "phase_3_pkl")
src_path = os.path.join(dir_path, '300ms_synced_29_03_01_06_may.pkl')
df.to_pickle(src_path)
print(df.shape)
print(df.head(20))

(2008037, 12)
                      time  brickVoltage001_BMS01  brickVoltage007_BMS01  \
0  2022-03-29 01:55:31.718                 4.1383                 4.1382   
1  2022-03-29 01:55:32.038                 4.1384                 4.1382   
2  2022-03-29 01:55:32.358                 4.1383                 4.1382   
3  2022-03-29 01:55:32.678                 4.1384                 4.1382   
4  2022-03-29 01:55:32.998                 4.1383                 4.1382   
5  2022-03-29 01:55:33.318                 4.1383                 4.1382   
6  2022-03-29 01:55:33.638                 4.1383                 4.1383   
7  2022-03-29 01:55:33.959                 4.1383                 4.1382   
8  2022-03-29 01:55:34.279                 4.1384                 4.1382   
9  2022-03-29 01:55:34.599                 4.1384                 4.1382   
10 2022-03-29 01:55:34.919                 4.1383                 4.1382   
11 2022-03-29 01:55:35.239                 4.1383                 4.1382  

In [105]:
# test voltage
querry = f"SELECT time, measure_value::double FROM DB_KELLY_v1.brickVoltage001_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-06 00:00:00' AND TIMESTAMP '2022-04-07 23:23:59' ORDER BY time ASC"
df_volt = wr.timestream.query(
            f"{querry}"
        )
value_colName = list(df_volt.columns)[1]

signal = 'brickVoltage001_BMS01'
# print(value_colName)
df_volt.rename(columns = {value_colName:signal}, inplace = True)
df_volt.time = pd.to_datetime(df_volt.time)

In [106]:
len(df_volt)
df_volt

# df_resample = df_volt.copy().resample("300ms").last().bfill(axis ='rows').reset_index()

,time,brickVoltage001_BMS01
0,2022-04-06 00:00:00.266,3.9583
1,2022-04-06 00:00:00.586,3.9584
2,2022-04-06 00:00:00.906,3.9583
3,2022-04-06 00:00:01.226,3.9583
4,2022-04-06 00:00:01.546,3.9582
...,...,...
106664,2022-04-07 22:02:01.389,4.0886
106665,2022-04-07 22:02:01.709,4.0886
106666,2022-04-07 22:02:02.029,4.0885
106667,2022-04-07 22:02:02.349,4.0885


In [107]:
# test current
querry = f"SELECT time, measure_value::double FROM DB_KELLY_v1.current_BMS01 WHERE time BETWEEN TIMESTAMP '2022-04-06 00:00:00' AND TIMESTAMP '2022-04-07 23:23:59' ORDER BY time ASC"
df_cur = wr.timestream.query(
            f"{querry}"
        )
value_colName = list(df_cur.columns)[1]

signal = 'current_BMS01'
# print(value_colName)
df_cur.rename(columns = {value_colName:signal}, inplace = True)
df_cur.time = pd.to_datetime(df_cur.time)

In [109]:
print(len(df_cur))
print(df_cur)

3413892
                           time  current_BMS01
0       2022-04-06 00:00:00.007            3.5
1       2022-04-06 00:00:00.017            3.5
2       2022-04-06 00:00:00.027            3.5
3       2022-04-06 00:00:00.039            3.5
4       2022-04-06 00:00:00.049            3.5
...                         ...            ...
3413887 2022-04-07 22:02:02.892            0.0
3413888 2022-04-07 22:02:02.902            0.0
3413889 2022-04-07 22:02:02.910            0.0
3413890 2022-04-07 22:02:02.922            0.0
3413891 2022-04-07 22:02:02.932            0.0

[3413892 rows x 2 columns]


In [110]:
merged_dataframe = pd.merge_asof(df_volt, df_cur, on="time", direction='nearest', tolerance=pd.Timedelta("300ms"),
                                 allow_exact_matches=False)

In [112]:
print(len(merged_dataframe))
print(merged_dataframe)

106669
                          time  brickVoltage001_BMS01  current_BMS01
0      2022-04-06 00:00:00.266                 3.9583            3.5
1      2022-04-06 00:00:00.586                 3.9584            3.5
2      2022-04-06 00:00:00.906                 3.9583            3.5
3      2022-04-06 00:00:01.226                 3.9583            3.5
4      2022-04-06 00:00:01.546                 3.9582            3.5
...                        ...                    ...            ...
106664 2022-04-07 22:02:01.389                 4.0886            0.0
106665 2022-04-07 22:02:01.709                 4.0886            0.0
106666 2022-04-07 22:02:02.029                 4.0885            0.0
106667 2022-04-07 22:02:02.349                 4.0885            0.0
106668 2022-04-07 22:02:02.669                 4.0886            0.0

[106669 rows x 3 columns]


In [89]:
print(df.head(3))

import sys, os
cwd = sys.path[0]

dir_path = os.path.join(cwd, 'csv', "phase_3_pkl")
print(dir_path)
src_path = os.path.join(dir_path, 'df.pkl')
print(src_path)
df.to_pickle(src_path)

                     time  current_BMS01  minSOC_BMS01  SOCocvInit_BMS01  \
0 2022-04-06 00:00:00.000            3.5           NaN               0.0   
1 2022-04-06 00:00:00.200            3.5         78.23               0.0   
2 2022-04-06 00:00:00.400            3.5         78.23               0.0   

   minSOH_BMS01  contactorState_BMS01  ctrOffTime_BMS01  \
0           NaN                   2.0               0.0   
1          96.5                   2.0               0.0   
2          96.5                   2.0               0.0   

   brickVoltage001_BMS01  moduleTemperature01_BMS01  
0                    NaN                        NaN  
1                 3.9583                       19.0  
2                 3.9584                       19.0  
/media/avt/development/ME_dev/battery_aging_algo/csv/phase_3_pkl
/media/avt/development/ME_dev/battery_aging_algo/csv/phase_3_pkl/df.pkl
